In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt

## First European City Example

In [3]:
paris_coords = np.array([
    [48.8575, 2.3514], #Center of Paris
    [48.8584, 2.2945], # Eiffel Tower
])

berlin_coords = np.array([
    [52.5200, 13.4050], # Center of Berlin
    [52.5163, 13.3777],   # Brandenburg Gate 
])

In [12]:
# Combine data into one matrix X and labels y
X_raw = np.vstack([paris_coords, berlin_coords]) 
X=X_raw[:,1:] #Just grab longitude
y = np.array([0, 0,  1, 1]) #Paris=0, Berlin=1

In [13]:
X

array([[ 2.3514],
       [ 2.2945],
       [13.405 ],
       [13.3777]])

In [14]:
y

array([0, 0, 1, 1])

In [18]:
class TinyGPSModel(torch.nn.Module):
    def __init__(self, input_size=1, output_size=2):
        super(TinyGPSModel, self).__init__()
        self.output = torch.nn.Linear(input_size, output_size) 

    def forward(self, x):
        x = self.output(x)
        return x

In [57]:
# Initialize model, loss, and optimizer
model = TinyGPSModel()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
num_steps=9

#Manually intialize model parameters
with torch.no_grad():
    model.output.weight[0,0]=-1.0
    model.output.weight[1,0]=1.0
    model.output.bias[0]=0.0
    model.output.bias[1]=0.0

weights=[]
grads=[]
xs=[]
ys=[]
logitss=[]
yhats=[]
losses=[]
accuracies=[]

# Training loop
for i in range(num_steps):    
    xs.append(X[i%len(y)])
    ys.append(y[i%len(y)])
    weights.append(np.concatenate([model.output.weight.detach().numpy().ravel(), model.output.bias.detach().numpy().ravel()]))
    
    optimizer.zero_grad()
    outputs = model(torch.tensor(X[i%len(y)]).float())
    loss = criterion(outputs, torch.tensor(y[i%len(y)])) 

    logitss.append(outputs.detach().numpy())
    yhats.append(torch.nn.Softmax(0)(outputs.detach()).numpy())
    
    loss.backward()  # backpropagation
    grads.append(np.concatenate([model.output.weight.grad.detach().numpy().ravel(), model.output.bias.grad.detach().numpy().ravel()]))
    losses.append(loss.item())
    optimizer.step() #
    
    with torch.no_grad():
        logits=model(torch.tensor(X, dtype=torch.float)) 
        accuracy=(torch.argmax(logits, dim=1)==torch.tensor(y)).sum().item()/len(y)
    print(f"Step {i+1}/{num_steps}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")
    accuracies.append(accuracy)

weights=np.array(weights)
grads=np.array(grads)
xs=np.array(xs)
ys=np.array(ys)
logitss=np.array(logitss)
yhats=np.array(yhats)
losses=np.array(losses)
accuracies=np.array(accuracies)

Step 1/9, Loss: 4.7118, Accuracy: 0.5000
Step 2/9, Loss: 3.3569, Accuracy: 0.5000
Step 3/9, Loss: 0.0000, Accuracy: 0.5000
Step 4/9, Loss: 0.0000, Accuracy: 0.5000
Step 5/9, Loss: 2.2819, Accuracy: 0.5000
Step 6/9, Loss: 1.2868, Accuracy: 0.5000
Step 7/9, Loss: 0.0222, Accuracy: 0.5000
Step 8/9, Loss: 0.0102, Accuracy: 0.5000
Step 9/9, Loss: 0.8456, Accuracy: 1.0000


In [65]:
import pandas as pd
fn='/Users/stephen/Stephencwelch Dropbox/welch_labs/ai_book/3_backprop_2/exercise_run_1.csv'
steps_to_show=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [66]:
df=pd.DataFrame([xs[steps_to_show, 0], 
                 weights[steps_to_show, 0], 
                 weights[steps_to_show, 1], 
                 grads[steps_to_show, 0], 
                 grads[steps_to_show, 1],
                 weights[steps_to_show, 2], 
                 weights[steps_to_show, 3], 
                 grads[steps_to_show, 2], 
                 grads[steps_to_show, 3],
                 logitss[steps_to_show, 0],
                 logitss[steps_to_show, 1],
                 yhats[steps_to_show, 0],
                 yhats[steps_to_show, 1],
                 ys[steps_to_show],
                 losses[steps_to_show],
                 accuracies[steps_to_show],
                 ], 
                 columns=steps_to_show
                 )

In [67]:
df

,0,1,2,3,4,5,6,7,8
0,2.351400,2.294500,1.340500e+01,1.337770e+01,2.351400,2.294500,13.405000,13.377700,2.351400
1,-1.000000,-0.766973,-5.455182e-01,-5.455191e-01,-0.545520,-0.334385,-0.168296,-0.197730,-0.211321
2,1.000000,0.766973,5.455182e-01,5.455192e-01,0.545520,0.334385,0.168296,0.197731,0.211321
3,-2.330265,-2.214553,8.820050e-06,9.067977e-06,-2.111351,-1.660891,0.294345,0.135907,-1.341903
4,2.330265,2.214553,-9.588003e-06,-9.568476e-06,2.111350,1.660891,-0.294346,-0.135907,1.341903
5,0.000000,0.099101,1.956169e-01,1.956168e-01,0.195617,0.285408,0.357794,0.355598,0.354582
6,0.000000,-0.099101,-1.956169e-01,-1.956168e-01,-0.195617,-0.285408,-0.357794,-0.355598,-0.354582
7,-0.991012,-0.965157,6.579671e-07,6.778428e-07,-0.897912,-0.723857,0.021958,0.010159,-0.570683
8,0.991012,0.965157,-7.152557e-07,-7.152557e-07,0.897912,0.723857,-0.021958,-0.010159,0.570683
9,-2.351400,-1.660720,-7.117055e+00,-7.102174e+00,-1.087119,-0.481838,-1.898212,-2.289580,-0.142318


In [69]:
df.round(3).to_csv(fn)

## Second Mini European City Example

In [89]:
paris_coords = np.array([
    [48.8575, 2.3514], #Center of Paris
    [48.8584, 2.2945], # Eiffel Tower
])

madrid_coords = np.array([
    [40.4167, -3.7033],   # Center of Madrid
    [40.4153, -3.6835],   # Retiro Park 
])

In [92]:
# Combine data into one matrix X and labels y
X_raw = np.vstack([paris_coords, madrid_coords]) 
X=X_raw[:,1:]
y = np.array([0, 0,  1, 1]) #Paris=0, Berlin=1

In [93]:
X

array([[ 2.3514],
       [ 2.2945],
       [-3.7033],
       [-3.6835]])

In [94]:
y

array([0, 0, 1, 1])

In [95]:
class TinyGPSModel(torch.nn.Module):
    def __init__(self, input_size=1, output_size=2):
        super(TinyGPSModel, self).__init__()
        self.output = torch.nn.Linear(input_size, output_size) 

    def forward(self, x):
        x = self.output(x)
        return x

In [103]:
# Initialize model, loss, and optimizer
model = TinyGPSModel()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
num_steps=4

#Manually intialize model parameters
with torch.no_grad():
    model.output.weight[0,0]=-1.0
    model.output.weight[1,0]=0.5
    model.output.bias[0]=0.0
    model.output.bias[1]=0.0

weights=[]
grads=[]
xs=[]
ys=[]
logitss=[]
yhats=[]
losses=[]
accuracies=[]

# Training loop
for i in range(num_steps):    
    xs.append(X[i%len(y)])
    ys.append(y[i%len(y)])
    weights.append(np.concatenate([model.output.weight.detach().numpy().ravel(), model.output.bias.detach().numpy().ravel()]))
    
    optimizer.zero_grad()
    outputs = model(torch.tensor(X[i%len(y)]).float())
    loss = criterion(outputs, torch.tensor(y[i%len(y)])) 

    logitss.append(outputs.detach().numpy())
    yhats.append(torch.nn.Softmax(0)(outputs.detach()).numpy())
    
    loss.backward()  # backpropagation
    grads.append(np.concatenate([model.output.weight.grad.detach().numpy().ravel(), model.output.bias.grad.detach().numpy().ravel()]))
    losses.append(loss.item())
    optimizer.step() #
    
    with torch.no_grad():
        logits=model(torch.tensor(X, dtype=torch.float)) 
        accuracy=(torch.argmax(logits, dim=1)==torch.tensor(y)).sum().item()/len(y)
    print(f"Step {i+1}/{num_steps}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")
    accuracies.append(accuracy)

weights=np.array(weights)
grads=np.array(grads)
xs=np.array(xs)
ys=np.array(ys)
logitss=np.array(logitss)
yhats=np.array(yhats)
losses=np.array(losses)
accuracies=np.array(accuracies)

Step 1/4, Loss: 3.5561, Accuracy: 0.0000
Step 2/4, Loss: 2.3044, Accuracy: 0.0000
Step 3/4, Loss: 2.7722, Accuracy: 1.0000
Step 4/4, Loss: 0.6685, Accuracy: 1.0000


In [107]:
import pandas as pd
fn='/Users/stephen/Stephencwelch Dropbox/welch_labs/ai_book/3_backprop_2/exercise_run_2.csv'
steps_to_show=[0, 1, 2]

In [108]:
df=pd.DataFrame([xs[steps_to_show, 0], 
                 weights[steps_to_show, 0], 
                 weights[steps_to_show, 1], 
                 grads[steps_to_show, 0], 
                 grads[steps_to_show, 1],
                 weights[steps_to_show, 2], 
                 weights[steps_to_show, 3], 
                 grads[steps_to_show, 2], 
                 grads[steps_to_show, 3],
                 logitss[steps_to_show, 0],
                 logitss[steps_to_show, 1],
                 yhats[steps_to_show, 0],
                 yhats[steps_to_show, 1],
                 ys[steps_to_show],
                 losses[steps_to_show],
                 accuracies[steps_to_show],
                 ], 
                 columns=steps_to_show
                 )

In [109]:
df.round(3).to_csv(fn)

## Linear Regression w/ MSE

In [110]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [183]:
# Create toy dataset: 4 points that follow y = 2x + 1
X = torch.tensor([[1.0], [2.0], [3.0], [4.0]], requires_grad=False)
y = torch.tensor([[3.0], [5.0], [7.0], [9.0]], requires_grad=False)

In [184]:
X

tensor([[1.],
        [2.],
        [3.],
        [4.]])

In [185]:
y

tensor([[3.],
        [5.],
        [7.],
        [9.]])

In [186]:
# Define the linear model y = mx + b
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)  # 1 input, 1 output
    
    def forward(self, x):
        return self.linear(x)


In [196]:
# Initialize model, loss function, and optimizer
model = LinearRegression()

with torch.no_grad():
    model.linear.weight[0,0]=1.0
    model.linear.bias[0]=0.0

criterion = nn.MSELoss()  # Mean Squared Error
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
num_steps = 8

weights=[]
grads=[]
xs=[]
ys=[]
yhats=[]
losses=[]

for i in range(num_steps):
    xs.append(X[i%len(y)])
    ys.append(y[i%len(y)])
    weights.append(np.concatenate([model.linear.weight.detach().numpy().ravel(), model.linear.bias.detach().numpy().ravel()]))
    
    # Forward pass
    y_pred = model(X[i%len(y)])
    loss = criterion(y_pred, y[i%len(y)])

    yhats.append(y_pred.detach().numpy())
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    grads.append(np.concatenate([model.linear.weight.grad.detach().numpy().ravel(), model.linear.bias.grad.detach().numpy().ravel()]))
    losses.append(loss.item())
    optimizer.step()

    print(f"Epoch [{i+1}/{num_steps}], Loss: {loss.item():.6f}")

weights=np.array(weights)
grads=np.array(grads)
xs=np.array(xs)
ys=np.array(ys)
yhats=np.array(yhats)
losses=np.array(losses)

Epoch [1/8], Loss: 4.000000
Epoch [2/8], Loss: 3.240000
Epoch [3/8], Loss: 0.014400
Epoch [4/8], Loss: 0.005184
Epoch [5/8], Loss: 0.020736
Epoch [6/8], Loss: 0.002304
Epoch [7/8], Loss: 0.013271
Epoch [8/8], Loss: 0.004777


In [197]:
weights

array([[1.        , 0.        ],
       [1.4       , 0.4       ],
       [2.12      , 0.76      ],
       [2.0479999 , 0.736     ],
       [2.1056004 , 0.7504001 ],
       [2.1344004 , 0.7792    ],
       [2.1152    , 0.76959985],
       [2.04608   , 0.74655986]], dtype=float32)

In [198]:
import pandas as pd
fn='/Users/stephen/Stephencwelch Dropbox/welch_labs/ai_book/3_backprop_2/exercise_run_3.csv'
steps_to_show=[0, 1, 2, 3, 4, 5, 6, 7]

In [205]:
df=pd.DataFrame([xs[steps_to_show, 0], 
                 weights[steps_to_show, 0], 
                 grads[steps_to_show, 0], 
                 weights[steps_to_show, 1], 
                 grads[steps_to_show, 1], 
                 yhats[steps_to_show, 0],
                 ys[steps_to_show].ravel(),
                 losses[steps_to_show],
                 ], 
                 columns=steps_to_show
                 )

In [206]:
df

,0,1,2,3,4,5,6,7
0,1.0,2.00,3.000000,4.000000,1.000000,2.000000,3.000000,4.000000
1,1.0,1.40,2.120000,2.048000,2.105600,2.134400,2.115200,2.046080
2,-4.0,-7.20,0.719999,-0.576004,-0.287999,0.192003,0.691200,-0.552956
3,0.0,0.40,0.760000,0.736000,0.750400,0.779200,0.769600,0.746560
4,-4.0,-3.60,0.240000,-0.144001,-0.287999,0.096002,0.230400,-0.138239
5,1.0,3.20,7.120000,8.927999,2.856000,5.048001,7.115200,8.930881
6,3.0,5.00,7.000000,9.000000,3.000000,5.000000,7.000000,9.000000
7,4.0,3.24,0.014400,0.005184,0.020736,0.002304,0.013271,0.004777


In [207]:
df.round(3).to_csv(fn)

In [208]:
weights

array([[1.        , 0.        ],
       [1.4       , 0.4       ],
       [2.12      , 0.76      ],
       [2.0479999 , 0.736     ],
       [2.1056004 , 0.7504001 ],
       [2.1344004 , 0.7792    ],
       [2.1152    , 0.76959985],
       [2.04608   , 0.74655986]], dtype=float32)

## Linear Regression with MAE

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [210]:
# Create toy dataset: 4 points that follow y = 2x + 1
X = torch.tensor([[1.0], [2.0], [3.0], [4.0]], requires_grad=False)
y = torch.tensor([[3.0], [5.0], [7.0], [9.0]], requires_grad=False)

In [221]:
# Initialize model, loss function, and optimizer
model = LinearRegression()

with torch.no_grad():
    model.linear.weight[0,0]=1.0
    model.linear.bias[0]=0.0

criterion = nn.L1Loss()  # Mean Squared Error
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
num_steps = 8

weights=[]
grads=[]
xs=[]
ys=[]
yhats=[]
losses=[]

for i in range(num_steps):
    xs.append(X[i%len(y)])
    ys.append(y[i%len(y)])
    weights.append(np.concatenate([model.linear.weight.detach().numpy().ravel(), model.linear.bias.detach().numpy().ravel()]))
    
    # Forward pass
    y_pred = model(X[i%len(y)])
    loss = criterion(y_pred, y[i%len(y)])

    yhats.append(y_pred.detach().numpy())
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    grads.append(np.concatenate([model.linear.weight.grad.detach().numpy().ravel(), model.linear.bias.grad.detach().numpy().ravel()]))
    losses.append(loss.item())
    optimizer.step()

    print(f"Epoch [{i+1}/{num_steps}], Loss: {loss.item():.6f}")

weights=np.array(weights)
grads=np.array(grads)
xs=np.array(xs)
ys=np.array(ys)
yhats=np.array(yhats)
losses=np.array(losses)

Epoch [1/8], Loss: 2.000000
Epoch [2/8], Loss: 2.700000
Epoch [3/8], Loss: 2.900000
Epoch [4/8], Loss: 2.300000
Epoch [5/8], Loss: 0.600000
Epoch [6/8], Loss: 0.300000
Epoch [7/8], Loss: 0.500000
Epoch [8/8], Loss: 0.500000


In [222]:
weights

array([[1.       , 0.       ],
       [1.1      , 0.1      ],
       [1.3000001, 0.2      ],
       [1.6      , 0.3      ],
       [2.       , 0.4      ],
       [2.1      , 0.5      ],
       [2.3      , 0.6      ],
       [2.       , 0.5      ]], dtype=float32)

In [223]:
import pandas as pd
fn='/Users/stephen/Stephencwelch Dropbox/welch_labs/ai_book/3_backprop_2/exercise_run_4.csv'
steps_to_show=[0, 1, 2, 3, 4, 5, 6, 7]

In [224]:
df=pd.DataFrame([xs[steps_to_show, 0], 
                 weights[steps_to_show, 0], 
                 grads[steps_to_show, 0], 
                 weights[steps_to_show, 1], 
                 grads[steps_to_show, 1], 
                 yhats[steps_to_show, 0],
                 ys[steps_to_show].ravel(),
                 losses[steps_to_show],
                 ], 
                 columns=steps_to_show
                 )

In [225]:
df.round(3).to_csv(fn)